In [127]:
debug = False

In [128]:
import numpy as np
import pandas as pd
from queue import Queue
import time

In [129]:
initial_position = list(pd.read_csv('./inicial_medium.csv', delimiter=';', header=None).values[0])

# posicion(0,1)
initial_position[1] = initial_position[1].replace(' ', '')
initial_position

['OOOLLL', '', 'I']

In [130]:
pd.read_csv('./inicial_medium.csv', delimiter=';', header=None)

,0,1,2
0,OOOLLL,,I


In [131]:
operations = [
    'O',  #oveja
    'L',  #lobo
    'OO',  #ovejas
    'OL',  #lobo y oveja
    'LL'  #lobos
]

In [132]:
class Node:
    def __init__(self, value, parent=None):
        self.coste = None
        self.value = value
        self.children = []
        self.parent = parent

    def add_child(self, child, coste = None):
        node = Node(child, parent=self)
        if coste is not None:
            node.coste = coste
        self.children.append(node)
        return node

    def __eq__(self, other):
        return self.coste == other.coste

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return self.coste < other.coste

    def __gt__(self, other):
        return self.coste > other.coste

    def __le__(self, other):
        return self.coste <= other.coste

    def __ge__(self, other):
        return self.coste >= other.coste


class Tree:
    def __init__(self, root):
        self.root = root


def find_path_to_root(objective_node):
    path = []
    current_node = objective_node
    while current_node is not None:
        path.insert(0, current_node.value)
        current_node = current_node.parent
    return path

In [133]:
def generate_children(estado, conjunto_de_operaciones):
    """
    Método que genera los posibles hijos de un estado dado
    Args:
        estado: Situación del problema.
        conjunto_de_operaciones: Operaciones que se pueden realizar

    Returns: Lista de posibles hijos (estados derivados del estado actual)

    """
    def change_position(operacion):
        """
        Método que cambia la posición de la barca
        Args:
            operacion: Animales que se quieren mover

        Returns: Nuevo estado del problema (animales a ambos lados y posición de la barca)


        """
        position = estado[2]
        if position == 'I':
            # Comprobar disponibilidad de animales. Si en el origen (actual) hay mas o mismos animales que los que se quieren mover se puede realizar la operación
            if all(estado[0].count(char) >= operacion.count(char) for char in set(operacion)):
                nueva_cadena_izquierda = estado[0]
                for char in operacion:
                    nueva_cadena_izquierda = nueva_cadena_izquierda.replace(char, '', 1)
                nuevo_destino_derecha = estado[1]
            else:
                if debug: print("No se puede realizar la operación {0} {1} {2}".format(estado, operacion, position))
                return None
        else:
            if all(estado[1].count(char) >= operacion.count(char) for char in set(operacion)):
                nueva_cadena_izquierda = estado[1]
                for char in operacion:
                    nueva_cadena_izquierda = nueva_cadena_izquierda.replace(char, '', 1)
                nuevo_destino_derecha = estado[0]
            else:
                if debug: print("No se puede realizar la operación {0} {1} {2}".format(estado, operacion, position))
                return None

        nuevo_destino_derecha = ''.join(sorted(nuevo_destino_derecha + operacion, reverse=True))

        # Comprobar estado válido
        if nueva_cadena_izquierda.count('L') > nueva_cadena_izquierda.count('O') > 0:
            return None
        if nuevo_destino_derecha.count('L') > nuevo_destino_derecha.count('O') > 0:
            return None

        # print("estado válido: ",nuevaCadena, ''.join(sorted(nuevoDestino + operation, reverse=True)), position)

        if position == 'I':
            return np.asarray([nueva_cadena_izquierda, nuevo_destino_derecha, 'D'])
        else:
            return np.asarray([nuevo_destino_derecha, nueva_cadena_izquierda, 'I'])

    children = []

    for op in conjunto_de_operaciones:
        new_position = change_position(op)
        if new_position is not None:
            children.append(new_position)

    return children

## Miscelánea

In [134]:
def movements_to_solution(solution_path):
    pasos = []

    for i in range(len(solution_path) - 1):
        estado_actual = solution_path[i]
        estado_siguiente = solution_path[i + 1]

        # Descubrir la dirección de la barca
        direccion = "-->" if estado_actual[2] == 'I' else "<--"

        # Descubrir qué animales se movieron
        if estado_actual[2] == 'I':
            # Animales desplazados de izquierda a derecha
            origen_antes = estado_actual[0]
            origen_despues = estado_siguiente[0]

            # Calcular diferencias
            ovejas_movidas = origen_antes.count('O') - origen_despues.count('O')
            lobos_movidos = origen_antes.count('L') - origen_despues.count('L')
        else:
            # Animales desplazados de derecha a izquierda
            origen_antes = estado_actual[1]
            origen_despues = estado_siguiente[1]

            ovejas_movidas = origen_antes.count('O') - origen_despues.count('O')
            lobos_movidos = origen_antes.count('L') - origen_despues.count('L')

        moved_animals = []
        if ovejas_movidas == 1:
            moved_animals.append("oveja")
        elif ovejas_movidas > 1:
            moved_animals.append(f"{ovejas_movidas} ovejas")

        if lobos_movidos == 1:
            moved_animals.append("lobo")
        elif lobos_movidos > 1:
            moved_animals.append(f"{lobos_movidos} lobos")

        pasos.append(
            "|".join(estado_actual[:2])
            + "|" + f"{', '.join(moved_animals)} {direccion}")

    return pasos

## Búsqueda en anchura

In [135]:
def construct_solution_BFS_river(tree):
    nodos_a_expandir = Queue()
    nodos_a_expandir.put(tree.root)
    nodos_visitados = []

    path = []

    while not nodos_a_expandir.empty():
        current_node = nodos_a_expandir.get()
        current_position = current_node.value

        if tuple(current_position) in nodos_visitados:
            continue

        nodos_visitados.append(tuple(current_position))

        if current_position[0] == "":
            current_node = current_node.add_child(current_position)
            path = find_path_to_root(current_node)
            return tree, path, list(nodos_visitados)



        children = generate_children(current_position, operations)

        for c in children:
            if debug: print(c)
            if tuple(c) not in nodos_visitados:
                if debug: print(c)
                child = current_node.add_child(c)
                nodos_a_expandir.put(child)


    return tree, [], nodos_visitados

In [136]:
start_time = time.time()


root = Tree(Node(initial_position))
arbol_obtenido, camino, nodos = construct_solution_BFS_river(root)

time.time() - start_time


0.0019974708557128906

In [137]:
print(nodos)

[('OOOLLL', '', 'I'), ('OOOLL', 'L', 'D'), ('OOLL', 'OL', 'D'), ('OOOL', 'LL', 'D'), ('OOOLL', 'L', 'I'), ('OOO', 'LLL', 'D'), ('OOOL', 'LL', 'I'), ('OL', 'OOLL', 'D'), ('OOLL', 'OL', 'I'), ('LL', 'OOOL', 'D'), ('LLL', 'OOO', 'I'), ('L', 'OOOLL', 'D'), ('OL', 'OOLL', 'I'), ('LL', 'OOOL', 'I'), ('', 'OOOLLL', 'D')]


In [138]:
movements = movements_to_solution(camino)

In [139]:
print(" Se consiguió la solución en {0} pasos.:".format(len(movements)-1))

print(movements)

 Se consiguió la solución en 11 pasos.:
['OOOLLL||oveja, lobo -->', 'OOLL|OL|oveja <--', 'OOOLL|L|2 lobos -->', 'OOO|LLL|lobo <--', 'OOOL|LL|2 ovejas -->', 'OL|OOLL|oveja, lobo <--', 'OOLL|OL|2 ovejas -->', 'LL|OOOL|lobo <--', 'LLL|OOO|2 lobos -->', 'L|OOOLL|oveja <--', 'OL|OOLL|oveja, lobo -->', '|OOOLLL| <--']


## Tiempos

In [143]:
import time


def medir_tiempo(algoritmo, posicion_inicial_testbench, numero_de_ejecuciones=100000):
    _debug=False
    tiempos = []
    for _ in range(numero_de_ejecuciones) :
        inicio = time.perf_counter()  # Tiempo inicial
        _root = Tree(Node(posicion_inicial_testbench))
        _arbol_obtenido, _camino, _nodos = algoritmo(_root)
        fin = time.perf_counter()  # Tiempo final
        tempo = fin - inicio  # Tiempo total
        tiempos.append(tempo)

    media = np.mean(tiempos)
    minimo = np.min(tiempos)
    maximo = np.max(tiempos)

    unidad_de_escalado = 1000 # 1 segundo = 1000 milisegundos

    return media * unidad_de_escalado, minimo * unidad_de_escalado, maximo * unidad_de_escalado

In [144]:
datos_BFS = medir_tiempo(construct_solution_BFS_river, initial_position)
# datos_A_estrella = medir_tempo(construct_solution_a_estrella_river, initial_position)
print("Algoritmo | Media {0: <10}| Mínimo {0: <9}| Máximo {0: <10} |".format(" "))
print("BFS       | {0:.10f} ms | {1:.10f} ms | {2:.10f} ms  |".format(datos_BFS[0], datos_BFS[1], datos_BFS[2]))
# print("A*        | {0:.10f} ms | {1:.10f} ms | {2:.10f} ms  |".format(datos_A_estrella[0], datos_A_estrella[1], datos_A_estrella[2]))


Algoritmo | Media           | Mínimo          | Máximo            |
BFS       | 0.6271787170 ms | 0.4696999968 ms | 165.7612000126 ms  |


In [145]:
def medir_nodos_generados(algoritmo, posicion_inicial_testbench, numero_de_ejecuciones=100000):
    """
    Mide el número de nodos generados de un algoritmo (BFS o A*).

    Args:
        algoritmo: Función del algoritmo a medir (construct_solution_BFS_river o construct_solution_a_estrella_river).
        posicion_inicial_testbench: Estado inicial do problema.

    Returns:
        nodos: Número de nodos generados.
    """
    nodos = []
    for _ in range(numero_de_ejecuciones) :
        _root = Tree(Node(posicion_inicial_testbench))
        _arbol_obtenido, _camino, _nodos = algoritmo(_root)
        nodos.append(len(_nodos))

    return np.mean(nodos)

In [146]:
nodos_BFS = medir_nodos_generados(construct_solution_BFS_river, initial_position)

In [147]:
nodos_BFS

15.0